# Download SmartSeq2 Expression Matrix as an Input to Scanpy

1. Go to ~/.config/hca/ and edit the config file
2. Change https://dss.data.humancellatlas.org/v1/swagger.json to https://dss.integration.data.humancellatlas.org/v1/swagger.json
3. Restart your computer
4. Execute the following code to get an analysis bundle. For some reason, you can only view bundles from the current day in raw without getting an error.

In [41]:
import hca.dss, json
client = hca.dss.DSSClient()

# query = {
#     "query": {
#       "bool": {
#         "must": [
#           {
#             "match": {
#               "files.process_json.processes.content.protocol_type.text": "analysis"
#             }
#           },
#           {
#             "range": {
#               "manifest.version": {
#                 "gte": "2018-07-12T100000.000000Z"
#               }
#             }
#           }
#         ]
#       }
#     }
#   }

query = {
    "query": {
      "bool": {
        "must": [
          {
            "query_string": {
              "fields": [
                "*uuid"
              ],
              "query": "703fcaa2-92f5-4f62-9173-adbec7c355c8"
            }
          },
          {
            "range": {
              "manifest.version": {
                "gte": "2018-07-12T100000.000000Z"
              }
            }
          }
        ]
      }
    }
  }

# Print all recent analysis bundles
# for bundle in client.post_search.iterate(es_query=query, replica='aws', output_format='raw'):
#     print(json.dumps(bundle, indent=4, sort_keys=True))
#     break

# Get the expression matrix
#file = client.get_file(replica='aws', uuid='703fcaa2-92f5-4f62-9173-adbec7c355c8')
file = client.get_file(replica='aws', uuid='c95ba558-b750-449d-97fc-eb367701a3a4')

with open('out.txt', 'w') as outfile:
    outfile.write(file.decode("utf-8"))
print('done')

done


In [43]:
print(client.post_search.__doc__)

Find bundles by searching their metadata with an Elasticsearch query



.. admonition:: Pagination

 This method supports pagination. Use ``DSSClient.post_search.iterate(**kwargs)`` to create a generator that
 yields all results, making multiple requests over the wire if necessary:

 .. code-block:: python

   for result in DSSClient.post_search.iterate(**kwargs):
       ...

 The keyword arguments for ``DSSClient.post_search.iterate()`` are identical to the arguments for
 ``DSSClient.post_search()`` listed here.
:param es_query:  Elasticsearch query 
:type es_query: typing.Mapping
:param output_format:  Specifies the output format. The default format, ``summary``, is a list of UUIDs for bundles that match the query. Set this parameter to ``raw`` to get the verbatim JSON metadata for bundles that match the query. 
:type output_format: typing.Union[str, NoneType]
:param replica:  Replica to search. 
:type replica: <class 'str'>
:param per_page:  Max number of results to return per page.

5. Decompress the .results file (our expression matrix) and write it to a .tsv file
6. Input the expression matrix to scanpy

In [ ]:
import csv

with open('out.txt', newline='') as tsvfile:
    reader = csv.DictReader(tsvfile, delimiter='\t')
    c = 0
    for row in reader:
        print(row['pme_FPKM'])
        c +=1
        if c > 100:
            break

In [35]:
import scanpy.api as sc

# print(sc.read_csv.__doc__)
# print(sc.read.__doc__)
# print(sc.read_text.__doc__)

adata = sc.read_csv(filename='test_exp_mtr.tsv', delimiter='\t', first_column_names=True)
#adata = anndata.AnnData(filename='expression_matrix.tsv', dtype=str)


ValueError: could not convert string to float: 'ENST00000373020.8,ENST00000494424.1,ENST00000496771.5,ENST00000612152.4,ENST00000614008.4'